# PEC model

Start by importing MagmaPEC and MagmaPandas and any other packages you want to use. Here we also import Pandas for importing pressure data. For details on the use of MagmaPandas, please see it's [documentation](https://magmapandas.readthedocs.io/en/latest/).

In [1]:
import MagmaPEC as mpc
import MagmaPandas as mp

import pandas as pd

Import your melt inclusion and olivine data. 

In [2]:
melt_file = "./data/melt.csv"
olivine_file = "./data/olivine.csv"

melt = mp.read_melt(melt_file, index_col=["name"])
olivine = mp.read_olivine(olivine_file, index_col=["name"])

If you have data for internal inclusion pressures (in bars) you can also import those

In [3]:
pressure_file ="./data/pressure.csv"

pressure = pd.read_csv(pressure_file, index_col = ["name"]).squeeze()

pressure

name
PI032-04-01    4104.410589
PI032-04-02    4231.966408
PI041-02-02    1956.244091
PI041-03-01    3548.843351
PI041-03-03    3500.514426
PI041-05-04    1386.274577
PI041-05-06    2512.013195
PI041-07-01    3668.361134
PI041-07-02    2449.116051
PI052-01-02    1404.218415
Name: P_bar, dtype: float64

Here we will also use a whole-rock dataset to set up a model for predicting the initial FeO contents of our melt inclusions. For explanation of this model, please follow the [Initial FeO prediction](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html#) example.

In [4]:
wholerock_file = "./data/wholerock.csv"

wholerock = mp.read_melt(wholerock_file, index_col=["name"])

Let's begin by setting up the model to predict the initial FeO content of our melt inclusions according to the previous [example](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html#). Here, the FeO prediction model is based on TiO<sub>2</sub>, Al<sub>2</sub>O<sub>3</sub> and CaO, which we can check by accessing their coefficients:

In [5]:
x = wholerock.drop(columns=["FeO"])
FeOi_predict = mpc.FeOi_prediction(x=x, FeO=wholerock["FeO"])

do_not_use = ["MnO", "P2O5", "Cr2O3", "total"]

model_fits = FeOi_predict.calculate_model_fits(exclude=do_not_use)
FeOi_predict.select_predictors(idx=3)

In [6]:
FeOi_predict.coefficients

intercept    11.585125
TiO2          1.463365
Al2O3        -0.230722
CaO          -0.169262
dtype: float64

To use this model during PEC correction, we need to store this model as a callable function, which we can do with the *model* attribute:

In [7]:
FeO_model = FeOi_predict.model

Let's first quickly test the model by predicting FeO contents for our uncorrected melt compositions:

In [8]:
FeO_model(melt)

name
PI032-04-01    10.196984
PI032-04-02    10.265411
PI041-02-02    10.221895
PI041-03-01    10.614069
PI041-03-03    10.716794
PI041-05-04     9.420381
PI041-05-06    11.476340
PI041-07-01    11.608516
PI041-07-02    11.451018
PI052-01-02     8.722012
dtype: float32

This all looks good, so let continue with setting up the PEC correction model.

First, preview the melt and olivine data to check everything looks ok:

In [9]:
melt.head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl
name,,,,,,,,,,,,,,,,
PI032-04-01,50.028862,15.226979,5.330006,10.522019,8.59454,3.914280,0.730798,0.126266,2.669216,0.303542,NaN,0.658252,1.508379,0.082590,0.152337,0.035565
PI032-04-02,49.558861,16.041588,5.117479,10.266067,8.25471,3.782408,0.997826,0.129969,2.814807,0.356900,NaN,0.713472,1.412274,0.089010,0.175511,0.047265
PI041-02-02,49.112045,16.972038,4.864153,9.195718,10.07540,3.793512,1.079184,0.153456,2.807967,0.561756,NaN,0.464520,0.656341,0.047715,0.068108,0.021185
PI041-03-01,47.098515,17.448515,4.650640,12.154110,7.95402,3.707263,1.268282,0.101370,3.493271,0.611504,NaN,0.883720,0.324968,0.088070,0.096228,0.059958
PI041-03-03,46.478870,17.637102,4.762752,12.364033,7.84027,3.789446,1.294603,0.077221,3.617484,0.574248,NaN,0.910470,0.344988,0.090605,0.088145,0.061584


In [10]:
olivine.head()

,SiO2,FeO,MgO,NiO,MnO,Al2O3,CaO,total
name,,,,,,,,
PI032-04-01,38.204800,15.981400,44.194000,0.172665,0.234256,0.005229,0.240552,99.032906
PI032-04-02,38.638500,15.898400,43.467400,0.188024,0.219599,-0.007310,0.234622,98.639240
PI041-02-02,37.270100,20.815901,41.015099,0.104744,0.293681,0.016547,0.214939,99.731000
PI041-03-01,38.795799,15.469300,44.750999,0.180198,0.212197,0.031333,0.259919,99.699745
PI041-03-03,38.701900,15.782500,44.920799,0.164372,0.214836,0.037263,0.275049,100.096720


Make sure that each row in *melt* and *olivine* is a matching pair of melt inclusion and olivine host. Here we use the sample names stored in the indices of both dataframes:

In [11]:
print(melt.index, "\n\n", olivine.index)

Index(['PI032-04-01', 'PI032-04-02', 'PI041-02-02', 'PI041-03-01',
       'PI041-03-03', 'PI041-05-04', 'PI041-05-06', 'PI041-07-01',
       'PI041-07-02', 'PI052-01-02'],
      dtype='object', name='name') 

 Index(['PI032-04-01', 'PI032-04-02', 'PI041-02-02', 'PI041-03-01',
       'PI041-03-03', 'PI041-05-04', 'PI041-05-06', 'PI041-07-01',
       'PI041-07-02', 'PI052-01-02'],
      dtype='object', name='name')


We can verify that the indices are the same with the *equals* method:

In [12]:
melt.index.equals(olivine.index)

True

Check the configuration of MagmaPandas and the PEC model and make changes if you want to (see the [configuration example](https://magmapec.readthedocs.io/en/latest/notebooks/config.html)). You can change MagmaPandas settings either via the MagmaPandas object (here: mp.configuration) or the MagmaPEC object (here: mpc.configuration).

In [13]:
print(mpc.configuration)
print(mpc.PEC_configuration)


################ MagmaPandas ################
#############################################
General settings_____________________________
fO2 buffer................................QFM
ΔfO2........................................1
Melt Fe3+/Fe2+........................Sun2024
Kd Fe-Mg ol-melt.......................toplis
Melt thermometer...............putirka2008_15
Volatile solubility model......IaconoMarziano
Volatile species........................mixed
#############################################


############ Post-entrapment crystallisation ############
################### correction model ####################
Settings_________________________________________________
Fe2+ behaviour...................................buffered
Stepsize equilibration (moles)...................0.002   
Stepsize crystallisation (moles).................0.05    
Decrease factor..................................5       
FeO convergence (wt. %)..........................0.05    
Kd convergence.............

Now we can initialise the PEC model, where we need the following data:

- **inclusions**

    inclusion major element compositions in oxide wt. % as a MagmaPandas Melt frame.

- **olivines**

    olivine major element compositions in oxide wt. % as a MagmaPandas Olivine frame.

- **P_bar**

    pressures in bar at which to run the model. You can use a fixed pressure for all inclusions, e.g. *P_bar=2e3* for 2 kbar, or indicate specific pressures per inclusion. In this example we do the latter, by passing the above imported pandas Series with internal inclusion pressures

- **FeO_target**

    Estimated initial FeO contents of melt inclusions. You can use a fixed value for all melt inclusions, e.g. FeO = 11, specific values for individual melt inclusions, stored in a pandas Series, or a predictive equation based on melt major element composition. This equation needs to be a callable function that accepts a Pandas DataFrame with melt compositions in oxide wt. % as input and return an array-like object with initial FeO contents per inclusion. In the example above we showed how to set up a function like this using MagmaPEC.

In [14]:
pec_model = mpc.PEC(inclusions=melt, olivines=olivine, P_bar=pressure, FeO_target=FeO_model)

Now we simply run the model with the *correct* method. It runs in two stages: the first stage equilibrates Fe and Mg between melt inclusions and olivine hosts through Fe-Mg cation exchange and the second stage corrects melt inclusions back to their initial FeO contents by melting or crystallising olivine.

This method returns three objects:

- corrected melt compositions as a MagmaPandas Melt frame

- PEC extents as a pandas DataFrame
    
- inclusion entrapment temperatures as a pandas Series

In [15]:
melts_corrected, pec, temperatures = pec_model.correct()


Equilibrating ... |██████████████████████████████| 100% [10/10] in 3.5s 
Correcting    ... |██████████████████████████████| 100% [10/10] in 13.4s 


The dataframe with pec results has three columns:

- equilibration_crystallisation: 
    
    crystallisation extents during the equilibration stage

- PE_crystallisation:

    crystallisation extents during the crystallisation stage

- total_crystallisation:

    total amount of post-entrapment crystallisation

with all data in percent.

In [16]:
pec

,equilibration_crystallisation,PE_crystallisation,total_crystallisation
name,,,
PI032-04-01,-2.589844,12.4,9.810583
PI032-04-02,-2.181641,13.0,10.817993
PI041-02-02,-0.404785,0.4,-0.004739
PI041-03-01,-1.751953,14.4,12.648291
PI041-03-03,-0.867676,13.0,12.132538
PI041-05-04,-1.602539,-2.8,-4.402173
PI041-05-06,-1.605469,3.6,1.994775
PI041-07-01,-2.976562,15.4,12.423010
PI041-07-02,-1.678711,12.8,11.121533


Results are also stored in the pec object and can be accessed via the *inclusions* and *olivine_corrected* attributes, while entrapment temperatures can be calculated on the fly with the *temperature* method of the *inclusions* Melt frame (see the [MagmaPandas documentation](https://magmapandas.readthedocs.io/en/latest/code_documentation.html#MagmaPandas.MagmaFrames.Melt.temperature))

In [17]:
pec_model.inclusions

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl,total
name,,,,,,,,,,,,,,,,,
PI032-04-01,48.932615,13.997643,7.685414,9.695639,10.306935,3.598133,0.671773,0.138911,2.453630,0.279025,0.000000,0.605086,1.386551,0.075919,0.140033,0.032693,100.0
PI032-04-02,48.478496,14.628181,7.549098,9.385885,10.342922,3.449319,0.909955,0.140868,2.566928,0.325471,0.000000,0.650642,1.287905,0.081172,0.160055,0.043102,100.0
PI041-02-02,49.151067,16.987807,4.818808,9.204932,10.173751,3.797024,1.080183,0.154553,2.810567,0.562276,0.000000,0.464950,0.656948,0.047759,0.068171,0.021204,100.0
PI041-03-01,45.917518,15.664650,7.210145,10.936913,10.702791,3.327537,1.138375,0.113616,3.135464,0.548870,0.000000,0.793203,0.291682,0.079049,0.086372,0.053817,100.0
PI041-03-03,45.331776,15.888064,7.027840,11.161822,10.799127,3.412885,1.165957,0.090181,3.258011,0.517185,0.000000,0.819995,0.310707,0.081601,0.079386,0.055464,100.0
PI041-05-04,47.862023,18.767805,3.607781,9.486345,9.277349,4.643880,1.622223,0.136675,2.510764,0.833209,0.000000,0.515007,0.466122,0.088609,0.122617,0.059592,100.0
PI041-05-06,46.422610,17.057541,4.662195,8.968911,11.465500,4.022334,1.424093,0.168945,3.652662,0.622665,0.000000,0.643353,0.588746,0.107968,0.128776,0.063701,100.0
PI041-07-01,45.783040,15.214560,7.317821,9.673379,11.600875,3.126866,1.270842,0.148978,3.528233,0.554192,0.000000,0.467155,1.019572,0.075009,0.163223,0.056256,100.0
PI041-07-02,45.865882,15.593893,6.879340,10.088148,11.463062,3.193845,1.361987,0.136368,3.539169,0.616881,0.000000,0.372679,0.617050,0.073673,0.140582,0.057441,100.0


In [18]:
pec_model.olivine_corrected

,equilibration_crystallisation,PE_crystallisation,total_crystallisation
name,,,
PI032-04-01,-2.589844,12.4,9.810583
PI032-04-02,-2.181641,13.0,10.817993
PI041-02-02,-0.404785,0.4,-0.004739
PI041-03-01,-1.751953,14.4,12.648291
PI041-03-03,-0.867676,13.0,12.132538
PI041-05-04,-1.602539,-2.8,-4.402173
PI041-05-06,-1.605469,3.6,1.994775
PI041-07-01,-2.976562,15.4,12.423010
PI041-07-02,-1.678711,12.8,11.121533


In [19]:
pec_model.inclusions.temperature(P_bar=pressure)

name
PI032-04-01    1476.797230
PI032-04-02    1475.724133
PI041-02-02    1404.583964
PI041-03-01    1476.801861
PI041-03-03    1472.908324
PI041-05-04    1374.397515
PI041-05-06    1410.102003
PI041-07-01    1474.333745
PI041-07-02    1463.516325
PI052-01-02    1376.773874
Name: T_K, dtype: float64